In [1]:
using JLD2
using Statistics
using Revise
using Plots 
using Laplacians

In [2]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [3]:
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic1 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic2 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic3 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic4 = copy(dic)

Dict{Any, Any} with 44 entries:
  "lamg_build"    => [Inf, Inf, Inf, 240.756, 446.312, 343.738, 341.245, Inf]
  "hypre_solve"   => [261.05, 99.43, Inf, 72.28, 188.39, 85.58, 158.43, 115.97]
  "icc2_tot"      => [926.774, 51.474, 62.579, 96.0824, 118.887, 20.404, 84.455…
  "lamg_its"      => [Inf, Inf, Inf, 9.0, 9.0, 8.0, 9.0, Inf]
  "lamg_solve"    => [Inf, Inf, Inf, 256.498, 544.379, 290.139, 399.051, Inf]
  "hypre_its"     => [27.0, 9.0, Inf, 9.0, 10.0, 9.0, 9.0, 9.0]
  "cmg_solve"     => [129.428, 161.038, 173.982, 96.0553, 215.098, 203.56, 168.…
  "ac_tot"        => [215.737, 232.003, 117.224, 196.134, 440.152, 306.406, 414…
  "cmg_build"     => [53.5463, 93.6143, 52.4846, 43.6402, 79.9469, 118.644, 82.…
  "icc2_solve"    => [910.359, 32.7137, 48.7333, 71.607, 102.014, 0.510183, 70.…
  "hypre_tot"     => [484.44, 349.89, Inf, 186.12, 528.59, 284.14, 406.41, 346.…
  "ac-s2m2_its"   => [33.0, 16.0, 21.0, 26.0, 27.0, 18.0, 34.0, 21.0]
  "icc_solve"     => [449.715, 77.3485, 23.1996, 9

In [4]:
nanErr!(dic1);
nanErr!(dic2);
nanErr!(dic3);
nanErr!(dic4);

In [5]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [6]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [7]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffK(d["nv"][1]), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for d in dics
        printOneLine(d, solvers)
    end
end

printAllLines (generic function with 1 method)

In [8]:
dics = [dic1, dic2, dic3, dic4];

In [9]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

9.54&103&0.375&0.385&0.431&0.537&0.592&0.714&8.68&10.5&20.7\\
97.9&105&0.606&0.701&1.43&0.909&1.11&1.82&2.06&3.02&Inf\\
990&23&1.4&1.59&2.16&1.66&2.14&2.42&2.36&3.13&17.3\\
9950&8&3.07&3.99&4.34&3.34&4.18&4.36&2.94&3.71&Inf\\


In [10]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

9.54&103&0.873&1.26&2.4&N/A&N/A&N/A&4.52&5.66&10.2\\
97.9&105&0.946&1.66&4.73&N/A&N/A&N/A&1.1&2.02&Inf\\
990&23&2.61&4.29&11.4\tnote{*}&N/A&N/A&N/A&1.96&4.35&31.6\\
9950&8&4.39&6.4&Inf&N/A&N/A&N/A&1.27&1.72&Inf\\
